# Profiling main skforecast classes and methods

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [4]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn


import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.utils import *

%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument


# Information system and libraries

In [5]:
# Versions
# ==============================================================================
print(f"Python version: {platform.python_version()}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"skforecast version: {skforecast.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"psutil version: {psutil.__version__}")
print("")

# Computer information
# ==============================================================================
#Computer network name
print(f"Computer network name: {platform.node()}")
#Machine type
print(f"Machine type: {platform.machine()}")
#Processor type
print(f"Processor type: {platform.processor()}")
#Platform type
print(f"Platform type: {platform.platform()}")
#Operating system
print(f"Operating system: {platform.system()}")
#Operating system release
print(f"Operating system release: {platform.release()}")
#Operating system version
print(f"Operating system version: {platform.version()}")
#Physical cores
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
#Logical cores
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.11.5
scikit-learn version: 1.4.2
skforecast version: 0.13.0
pandas version: 2.2.2
numpy version: 1.26.2
scipy version: 1.11.4
psutil version: 5.9.6

Computer network name: ITES015-NB0029
Machine type: AMD64
Processor type: Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
Platform type: Windows-10-10.0.19045-SP0
Operating system: Windows
Operating system release: 10
Operating system version: 10.0.19045
Number of physical cores: 8
Number of logical cores: 16


# Data

In [18]:
# Data
# ==============================================================================
len_series = 1_000
n_series = 1_000
n_exog = 300
rgn = np.random.default_rng(seed=123)
y = pd.Series(
    rgn.random(size=(len_series)),
    name="y",
    index=pd.date_range(start="2000-01-01", periods=len_series, freq="h"),
)
exog = pd.DataFrame(rgn.random(size=(len_series, n_exog)), index=y.index)
exog.columns = [f"exog_{i}" for i in range(exog.shape[1])]
series = pd.DataFrame(rgn.random(size=(len_series, n_series)), index=y.index)
series.columns = [f"series_{i+1}" for i in range(series.shape[1])]
exog_test = exog.copy()
exog_test.index = exog.index + pd.DateOffset(hours=len_series)
exog_test = exog_test.asfreq("h")
print(f"Shape of y: {y.shape}")
print(f"Shape of exog: {exog.shape}")
print(f"Shape of series: {series.shape}")
print(f"Shape of exog_test: {exog_test.shape}")

Shape of y: (1000,)
Shape of exog: (1000, 300)
Shape of series: (1000, 1000)
Shape of exog_test: (1000, 300)


# ForecasterAutoregMultiSeries

In [7]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 24,
                 encoding  = 'ordinal',
                 transformer_exog=None,
                 transformer_series=None
             )

c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


In [8]:
%%pyinstrument

forecaster.fit(series=series, exog=exog)

_     ._   __/__   _ _  _  _ _/_   Recorded: 00:29:51  Samples:  2848
 /_//_/// /_\ / //_// / //_'/ //     Duration: 9.047     CPU time: 16.344
/   _/                      v4.6.2

Program: c:\Users\jaesc2\Miniconda3\envs\skforecast_py11\Lib\site-packages\ipykernel_launcher.py --f=c:\Users\jaesc2\AppData\Roaming\jupyter\runtime\kernel-v2-11608QYXNX2FdyLNU.json

9.046 <module>  ..\..\AppData\Local\Temp\ipykernel_16584\3979206768.py:1
|- 8.955 ForecasterAutoregMultiSeries.fit  skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:1052
|  |- 3.217 wrapper  sklearn\base.py:1457
|  |  `- 3.155 Ridge.fit  sklearn\linear_model\_ridge.py:1145
|  |     |- 2.167 Ridge.fit  sklearn\linear_model\_ridge.py:850
|  |     |  |- 1.471 _preprocess_data  sklearn\linear_model\_base.py:109
|  |     |  |  |- 0.834 check_array  sklearn\utils\validation.py:718
|  |     |  |  |  |- 0.495 _asarray_with_order  sklearn\utils\_array_api.py:501
|  |     |  |  |  |  `- 0.495 array  <built-in>
|  |     |  |  |  `- 0.339 _assert_all_finite  sklearn\utils\validation.py:96
|  |     |  |  |     `- 0.339 sum  numpy\core\fromnumeric.py:2177
|  |     |  |  |        `- 0.339 _wrapreduction  numpy\core\fromnumeric.py:71
|  |     |  |  |           `- 0.339 ufunc.reduce  <built-in>
|  |     |  |  |- 0.364 average  numpy\lib\function_base.py:398
|  |     |  |  |  `- 0.364 _mean  numpy\core\_methods.py:101
|  |     |  |  |     `- 0.364 ufunc.reduce  <built-in>
|  |     |  |  `- 0.273 [self]  sklearn\linear_model\_base.py
|  |     |  `- 0.696 _ridge_regression  sklearn\linear_model\_ridge.py:586
|  |     |     `- 0.696 _solve_cholesky  sklearn\linear_model\_ridge.py:192
|  |     |        `- 0.689 safe_sparse_dot  sklearn\utils\extmath.py:164
|  |     |- 0.822 Ridge._validate_data  sklearn\base.py:537
|  |     |  `- 0.821 check_X_y  sklearn\utils\validation.py:1124
|  |     |     `- 0.820 check_array  sklearn\utils\validation.py:718
|  |     |        |- 0.483 _asarray_with_order  sklearn\utils\_array_api.py:501
|  |     |        |  `- 0.483 DataFrame.__array__  pandas\core\generic.py:2149
|  |     |        |     `- 0.483 DataFrame._values  pandas\core\frame.py:1111
|  |     |        |        `- 0.483 DataFrame.values  pandas\core\frame.py:12590
|  |     |        |           `- 0.483 BlockManager.as_array  pandas\core\internals\managers.py:1633
|  |     |        |              `- 0.483 BlockManager._interleave  pandas\core\internals\managers.py:1707
|  |     |        `- 0.332 _assert_all_finite  sklearn\utils\validation.py:96
|  |     |           `- 0.332 sum  numpy\core\fromnumeric.py:2177
|  |     |              `- 0.332 _wrapreduction  numpy\core\fromnumeric.py:71
|  |     |                 `- 0.332 ufunc.reduce  <built-in>
|  |     `- 0.167 [self]  sklearn\linear_model\_ridge.py
|  |- 3.060 ForecasterAutoregMultiSeries._create_train_X_y  skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:595
|  |  |- 1.190 concat  pandas\core\reshape\concat.py:157
|  |  |  `- 1.171 _Concatenator.get_result  pandas\core\reshape\concat.py:622
|  |  |     `- 1.095 concatenate_managers  pandas\core\internals\concat.py:94
|  |  |        |- 0.553 _maybe_reindex_columns_na_proxy  pandas\core\internals\concat.py:202
|  |  |        |  `- 0.553 BlockManager.copy  pandas\core\internals\managers.py:557
|  |  |        |     `- 0.553 BlockManager.apply  pandas\core\internals\managers.py:317
|  |  |        |        `- 0.553 NumpyBlock.copy  pandas\core\internals\blocks.py:790
|  |  |        |           `- 0.553 ndarray.copy  <built-in>
|  |  |        |- 0.397 _concat_homogeneous_fastpath  pandas\core\internals\concat.py:249
|  |  |        `- 0.100 _get_combined_plan  pandas\core\internals\concat.py:296
|  |  |           `- 0.093 <listcomp>  pandas\core\internals\concat.py:303
|  |  |              `- 0.092 [self]  pandas\core\internals\concat.py
|  |  |- 0.622 ForecasterAutoregMultiSeries._create_train_X_y_single_series  skforecast\ForecasterAutoregMultiSeries\Fore

In [7]:
def prof_function(forecaster):
    forecaster.fit(series=series, exog=exog)

%prun prof_function(forecaster)

         3569293 function calls (3509962 primitive calls) in 14.515 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    2.882    0.961    2.882    0.961 extmath.py:160(safe_sparse_dot)
        3    1.219    0.406    1.220    0.407 managers.py:1707(_interleave)
     1006    0.945    0.001    1.040    0.001 numeric.py:2378(array_equal)
     8035    0.777    0.000    0.777    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     1009    0.743    0.001    0.743    0.001 {method 'copy' of 'numpy.ndarray' objects}
        1    0.729    0.729    0.730    0.730 concat.py:249(_concat_homogeneous_fastpath)
     6022    0.679    0.000    0.679    0.000 {built-in method numpy.array}
1002/1001    0.647    0.001    0.648    0.001 {built-in method _operator.eq}
        1    0.283    0.283    1.195    1.195 _base.py:189(_preprocess_data)
     2005    0.233    0.000    0.237    0.000 missing.py:261(_isna_array)
717364/710279    0.2

In [8]:
series_indexes = {series[col].name: series[col].index for col in series.columns}
exog_dict = exog.to_dict(orient='series')

def funt_to_profile(forecaster, series, exog):
    forecaster._create_train_X_y(series=series, exog=exog)

%lprun -f forecaster._create_train_X_y funt_to_profile(forecaster, series, exog)

Timer unit: 1e-09 s

Total time: 5.4734 s
File: /home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py
Function: _create_train_X_y at line 595

Line #      Hits         Time  Per Hit   % Time  Line Contents
   595                                               def _create_train_X_y(
   596                                                   self,
   597                                                   series: Union[pd.DataFrame, dict],
   598                                                   exog: Optional[Union[pd.Series, pd.DataFrame, dict]]=None,
   599                                                   store_last_window: Union[bool, list]=True,
   600                                               ) -> Tuple[pd.DataFrame, pd.Series, dict, list, list, list, dict, dict]:
   601                                                   """
   602                                                   Create training matrices from multiple time series and

In [9]:
import pandas as pd
import numpy as np
import time


# Original code encapsulated in a function
def check_dtypes_original(exog_dict, exog_col_names):
    exog_dtype_dict = {col_name: set() 
                       for col_name in exog_col_names}
    for v in exog_dict.values():
        if v is not None:
            for col_name in v.columns:
                exog_dtype_dict[col_name].add(v[col_name].dtype.name)

    for col_name, dtypes in exog_dtype_dict.items():
        if len(dtypes) > 1:
            raise TypeError(
                (f"Column '{col_name}' has different dtypes in different exog "
                 f"DataFrames or Series.")
            )

# Optimized code encapsulated in a function
def check_dtypes_optimized_1(exog_dict):
    first_dtype_dict = {}
    for v in exog_dict.values():
        if v is not None:
            for col_name in v.columns:
                current_dtype = v[col_name].dtype.name
                if col_name not in first_dtype_dict:
                    first_dtype_dict[col_name] = current_dtype
                elif first_dtype_dict[col_name] != current_dtype:
                    raise TypeError(f"Column '{col_name}' has different dtypes in different exog DataFrames or Series.")
                

def check_dtypes_optimized_2(exog_dict):
    exog_dtype_dict = {}
    for df in exog_dict.values():
        if df is not None:
            for col_name, dtype in df.dtypes.items():
                if col_name not in exog_dtype_dict:
                    exog_dtype_dict[col_name] = set()
                exog_dtype_dict[col_name].add(dtype.name)
                if len(exog_dtype_dict[col_name]) > 1:
                    raise TypeError(f"Column '{col_name}' has different dtypes in different exog DataFrames or Series.")



def check_dtypes_optimized_3(exog_dict):
    exog_dtypes_buffer = []
    for df in exog_dict.values():
        if df is not None:
            exog_dtypes_buffer.append(df.dtypes)
    exog_dtypes_buffer = pd.concat(exog_dtypes_buffer, axis=1)
    if not (exog_dtypes_buffer.nunique(axis=1, dropna=False) == 1).all():
        raise TypeError("Some columns have different dtypes in different exog DataFrames or Series.")
    

def check_dtypes_optimized_4(exog_dict):
    exog_dtypes_buffer = []
    for df in exog_dict.values():
        if df is not None:
            exog_dtypes_buffer.append(df.dtypes)
    exog_dtypes_buffer = np.concatenate(exog_dtypes_buffer)
    if not np.all(exog_dtypes_buffer == exog_dtypes_buffer[0]):
        raise TypeError("Some columns have different dtypes in different exog DataFrames or Series.")


# No error
# ------------------------------------------------------------------------------
n_exog = 30
n_series = 1000
df = pd.DataFrame(np.random.rand(1000, n_exog), columns=[f'col_{i}' for i in range(n_exog)])
exog_dict = {f"df_{i}": df.copy() for i in range(n_series)}
exog_col_names = df.columns

start_time = time.time()
try:
    check_dtypes_original(exog_dict, exog_col_names)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Original code execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_1(exog_dict)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 1 execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_2(exog_dict)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 2 execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_3(exog_dict)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 3 execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_4(exog_dict)
except TypeError as e:  
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 4 execution time: {end_time - start_time} seconds")
print("")


# With error
# ------------------------------------------------------------------------------
n_exog = 30
n_series = 1000
df = pd.DataFrame(np.random.rand(1000, n_exog), columns=[f'col_{i}' for i in range(n_exog)])
exog_dict = {f"df_{i}": df.copy() for i in range(n_series)}
exog_dict["df_100"]['col_13'] = exog_dict["df_100"]['col_13'].astype('str')
exog_col_names = df.columns

start_time = time.time()
try:
    check_dtypes_original(exog_dict, exog_col_names)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Original code execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_1(exog_dict)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 1 execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_2(exog_dict)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 2 execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_3(exog_dict)
except TypeError as e:
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 3 execution time: {end_time - start_time} seconds")

start_time = time.time()
try:
    check_dtypes_optimized_4(exog_dict)
except TypeError as e:  
    end_time = time.time()
    print(e)
finally:
    end_time = time.time()
print(f"Optimized code 4 execution time: {end_time - start_time} seconds")

Original code execution time: 1.572906732559204 seconds
Optimized code 1 execution time: 0.3028547763824463 seconds
Optimized code 2 execution time: 0.21156692504882812 seconds
Optimized code 3 execution time: 0.10552453994750977 seconds
Optimized code 4 execution time: 0.0680389404296875 seconds

Column 'col_13' has different dtypes in different exog DataFrames or Series.
Original code execution time: 1.5326159000396729 seconds
Column 'col_13' has different dtypes in different exog DataFrames or Series.
Optimized code 1 execution time: 0.030980348587036133 seconds
Column 'col_13' has different dtypes in different exog DataFrames or Series.
Optimized code 2 execution time: 0.02108454704284668 seconds
Some columns have different dtypes in different exog DataFrames or Series.
Optimized code 3 execution time: 0.10317015647888184 seconds
Some columns have different dtypes in different exog DataFrames or Series.
Optimized code 4 execution time: 0.0688016414642334 seconds


## Predict

In [24]:
def funt_to_profile(forecaster, exog_test):
    forecaster.predict(steps=10, exog=exog_test)

%lprun -f forecaster.predict funt_to_profile(forecaster, exog_test)

Timer unit: 1e-07 s

Total time: 2.29601 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py
Function: predict at line 1464

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1464                                               def predict(
  1465                                                   self,
  1466                                                   steps: int,
  1467                                                   levels: Optional[Union[str, list]]=None,
  1468                                                   last_window: Optional[pd.DataFrame]=None,
  1469                                                   exog: Optional[Union[pd.Series, pd.DataFrame, dict]]=None,
  1470                                                   suppress_warnings: bool=False
  1471                                               ) -> pd.DataFrame:
  1472                                                   """
  1473             

In [25]:
def funt_to_profile(forecaster, exog_test):
    forecaster.create_predict_inputs(steps=10, exog=exog_test)

%lprun -f forecaster.create_predict_inputs funt_to_profile(forecaster, exog_test)

Timer unit: 1e-07 s

Total time: 0.30582 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py
Function: create_predict_inputs at line 1203

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1203                                               def create_predict_inputs(
  1204                                                   self,
  1205                                                   steps: int,
  1206                                                   levels: Optional[Union[str, list]]=None,
  1207                                                   last_window: Optional[pd.DataFrame]=None,
  1208                                                   exog: Optional[Union[pd.Series, pd.DataFrame, dict]]=None,
  1209                                                   predict_boot: bool=False,
  1210                                                   in_sample_residuals: bool=True,
  1211                                 

In [29]:
from skforecast.utils import prepare_levels_multiseries

levels, input_levels_is_list = prepare_levels_multiseries(
                                    series_X_train = forecaster.series_X_train,
                                         levels         = None) 
levels[:2]

['series_1', 'series_10']

In [32]:
from skforecast.utils import preprocess_levels_self_last_window_multiseries

def funt_to_profile(forecaster, levels, input_levels_is_list):
    preprocess_levels_self_last_window_multiseries(
        levels=levels, input_levels_is_list=input_levels_is_list, last_window=forecaster.last_window
    )

%lprun -f preprocess_levels_self_last_window_multiseries funt_to_profile(forecaster, levels, input_levels_is_list)

Timer unit: 1e-07 s

Total time: 0.188799 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py
Function: preprocess_levels_self_last_window_multiseries at line 2595

Line #      Hits         Time  Per Hit   % Time  Line Contents
  2595                                           def preprocess_levels_self_last_window_multiseries(
  2596                                               levels: list,
  2597                                               input_levels_is_list: bool,
  2598                                               last_window: dict
  2599                                           ) -> Tuple[list, pd.DataFrame]:
  2600                                               """
  2601                                               Preprocess `levels` and `last_window` arguments in multiseries Forecasters. 
  2602                                               Only levels whose last window ends at the same datetime index will 
  2603                                         

In [41]:
inputs = forecaster.create_predict_inputs(steps=10, exog=exog_test, output_type='numpy')

In [43]:
def funt_to_profile(forecaster, inputs):
    forecaster._recursive_predict(
        steps=10, level='series_1', last_window=inputs[0]['series_1'], exog=inputs[1]['series_1']
    )

%lprun -f forecaster._recursive_predict funt_to_profile(forecaster, inputs)

Timer unit: 1e-07 s

Total time: 0.0042027 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py
Function: _recursive_predict at line 1402

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1402                                               def _recursive_predict(
  1403                                                   self,
  1404                                                   steps: int,
  1405                                                   level: str,
  1406                                                   last_window: np.ndarray,
  1407                                                   exog: Optional[np.ndarray]=None
  1408                                               ) -> np.ndarray:
  1409                                                   """
  1410                                                   Predict n steps ahead. It is an iterative process in which, each prediction,
  1411                 

In [46]:
%%pyinstrument

_ = forecaster.predict_interval(steps=15, levels=['series_1', 'series_2', 'series_3'], exog=exog_test, interval=[0.1, 0.9])

_     ._   __/__   _ _  _  _ _/_   Recorded: 00:58:23  Samples:  2753
 /_//_/// /_\ / //_// / //_'/ //     Duration: 2.756     CPU time: 2.750
/   _/                      v4.6.2

Program: c:\Users\jaesc2\Miniconda3\envs\skforecast_py11\Lib\site-packages\ipykernel_launcher.py --f=c:\Users\jaesc2\AppData\Roaming\jupyter\runtime\kernel-v2-11608QYXNX2FdyLNU.json

2.755 <module>  ..\..\AppData\Local\Temp\ipykernel_16584\847355136.py:1
`- 2.755 ForecasterAutoregMultiSeries.predict_interval  skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:1710
   `- 2.744 ForecasterAutoregMultiSeries.predict_bootstrapping  skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:1556
      |- 2.484 ForecasterAutoregMultiSeries._recursive_predict  skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:1402
      |  |- 1.873 Ridge.predict  sklearn\linear_model\_base.py:272
      |  |  `- 1.864 Ridge._decision_function  sklearn\linear_model\_base.py:266
      |  |     |- 1.548 Ridge._validate_data  sklearn\base.py:537
      |  |     |  |- 1.334 check_array  sklearn\utils\validation.py:718
      |  |     |  |  |- 0.766 _assert_all_finite  sklearn\utils\validation.py:96
      |  |     |  |  |  |- 0.203 _NumPyAPIWrapper.isdtype  sklearn\utils\_array_api.py:327
      |  |     |  |  |  |  `- 0.194 isdtype  sklearn\utils\_array_api.py:123
      |  |     |  |  |  |     |- 0.160 <genexpr>  sklearn\utils\_array_api.py:130
      |  |     |  |  |  |     |  `- 0.147 _isdtype_single  sklearn\utils\_array_api.py:135
      |  |     |  |  |  |     |     `- 0.126 supported_float_dtypes  sklearn\utils\_array_api.py:170
      |  |     |  |  |  |     |        |- 0.065 [self]  sklearn\utils\_array_api.py
      |  |     |  |  |  |     |        `- 0.057 _NumPyAPIWrapper.__getattr__  sklearn\utils\_array_api.py:272
      |  |     |  |  |  |     |           `- 0.050 [self]  sklearn\utils\_array_api.py
      |  |     |  |  |  |     `- 0.032 [self]  sklearn\utils\_array_api.py
      |  |     |  |  |  |- 0.164 [self]  sklearn\utils\validation.py
      |  |     |  |  |  |- 0.143 sum  numpy\core\fromnumeric.py:2177
      |  |     |  |  |  |  |- 0.111 _wrapreduction  numpy\core\fromnumeric.py:71
      |  |     |  |  |  |  |  `- 0.065 ufunc.reduce  <built-in>
      |  |     |  |  |  |  `- 0.030 [self]  numpy\core\fromnumeric.py
      |  |     |  |  |  |- 0.089 errstate.__enter__  numpy\core\_ufunc_config.py:430
      |  |     |  |  |  |  `- 0.079 seterr  numpy\core\_ufunc_config.py:33
      |  |     |  |  |  |     |- 0.040 [self]  numpy\core\_ufunc_config.py
      |  |     |  |  |  |     `- 0.028 geterr  numpy\core\_ufunc_config.py:132
      |  |     |  |  |  `- 0.075 errstate.__exit__  numpy\core\_ufunc_config.py:435
      |  |     |  |  |     `- 0.057 seterr  numpy\core\_ufunc_config.py:33
      |  |     |  |  |        `- 0.028 geterr  numpy\core\_ufunc_config.py:132
      |  |     |  |  |- 0.166 [self]  sklearn\utils\validation.py
      |  |     |  |  |- 0.131 _num_samples  sklearn\utils\validation.py:364
      |  |     |  |  |  `- 0.068 [self]  sklearn\utils\validation.py
      |  |     |  |  |- 0.074 _asarray_with_order  sklearn\utils\_array_api.py:501
      |  |     |  |  |  `- 0.040 _is_numpy_namespace  sklearn\utils\_array_api.py:111
      |  |     |  |  |- 0.060 simplefilter  warnings.py:165
      |  |     |  |  |  `- 0.042 _add_filter  warnings.py:181
      |  |     |  |  `- 0.030 get_namespace  sklearn\utils\_array_api.py:334
      |  |     |  |- 0.077 Ridge._check_feature_names  sklearn\base.py:448
      |  |     |  |- 0.073 Ridge._check_n_features  sklearn\base.py:402
      |  |     |  |  `- 0.052 _num_features  sklearn\utils\validation.py:311
      |  |     |  |     `- 0.042 [self]  sklearn\utils\validation.py
      |  |     |  `- 0.058 [self]  sklearn\base.py
      |  |     |- 0.189 check_is_fitted  sklearn\utils\validation.py:1548
      |  |     |  `- 0.155 _is_fitted  sklearn\utils\validation.py:1510
      |  |  